Part of course work Data Analytics for cyber security 

In [ ]:
# import the neural network from PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# create a class whose parent is the nn.Module class
class Net(nn.Module):
   def __init__(self):
      super(Net, self).__init__() # call the superclass constructor
      # create a network with one hidden layer (2 inputs, 2 outputs) and one output layer (2x1)
      self.hidden = nn.Linear(2,2)
      self.output = nn.Linear(2,1)

   # we need to define how data moves forward through the network
   # the back propagation is done automatically
   def forward(self,x):
      x = torch.sigmoid(self.hidden(x))
      x = torch.sigmoid(self.output(x))
      return x

net = Net()
if torch.cuda.is_available():
   net = net.to('cuda')
print(net)

# note, we could have done this whole thing with net = nn.Sequential(nn.Linear(2,2),nn.Sigmoid(), nn.Linear(2,1), nn.Sigmoid())

Net(
  (hidden): Linear(in_features=2, out_features=2, bias=True)
  (output): Linear(in_features=2, out_features=1, bias=True)
)


In [ ]:
from torch.utils.data import Dataset
class SampleDataset(Dataset):
   def __init__(self):
      # We are going to attempt to learn the first quadrant
      num_samples = 1000
      self.training = torch.randint(-100,100,(num_samples,2)).float()
      self.desired = torch.tensor([[1.0] if self.training[j,0]>0 and self.training[j,1]>0 else [0.0] for j in range(num_samples)])
   def __len__(self):
      return len(self.training)

   def __getitem__(self,idx):
      return((self.training[idx],self.desired[idx]))
dataset=SampleDataset()


In [ ]:
import torch.optim as optim
import time

optimizer = optim.SGD(net.parameters(), lr=0.01)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=40, shuffle=True)

num_epochs = 2000
start_time = time.clock()
for epoch in range(num_epochs):
   running_loss = 0.0
   for i,data in enumerate(train_loader):
      training,desired = data
      if torch.cuda.is_available():
         training = training.to('cuda')
         desired = desired.to('cuda')
      # need to zero out gradients for each epoch
      optimizer.zero_grad()
      #print(training,training.device)
      outputs = net(training)
      loss = F.mse_loss(outputs,desired)
      loss.backward() # compute the partial derivatives
      optimizer.step() # update the weights
      running_loss += loss.item()
   if epoch%40 == 39: # print every 40 times
      print('[%d] loss: %.3f'%(epoch,running_loss/10))
      running_loss = 0.0
end_time = time.clock()
print(end_time-start_time)  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


[39] loss: 0.417
[79] loss: 0.375
[119] loss: 0.352
[159] loss: 0.335
[199] loss: 0.314
[239] loss: 0.259
[279] loss: 0.227
[319] loss: 0.204
[359] loss: 0.186
[399] loss: 0.170
[439] loss: 0.157
[479] loss: 0.146
[519] loss: 0.137
[559] loss: 0.128
[599] loss: 0.120
[639] loss: 0.113
[679] loss: 0.107
[719] loss: 0.101
[759] loss: 0.096
[799] loss: 0.091
[839] loss: 0.087
[879] loss: 0.083
[919] loss: 0.079
[959] loss: 0.076
[999] loss: 0.073
[1039] loss: 0.070
[1079] loss: 0.067
[1119] loss: 0.065
[1159] loss: 0.062
[1199] loss: 0.060
[1239] loss: 0.058
[1279] loss: 0.056
[1319] loss: 0.054
[1359] loss: 0.053
[1399] loss: 0.051
[1439] loss: 0.049
[1479] loss: 0.048
[1519] loss: 0.047
[1559] loss: 0.045
[1599] loss: 0.044
[1639] loss: 0.043
[1679] loss: 0.042
[1719] loss: 0.041
[1759] loss: 0.040
[1799] loss: 0.039
[1839] loss: 0.038
[1879] loss: 0.037
[1919] loss: 0.036
[1959] loss: 0.035
[1999] loss: 0.034
68.86542


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [ ]:
num_tests=2000
testing = torch.randint(-100,100,(num_tests,2)).float()
print(testing)
desired = torch.tensor([1.0 if testing[j,0]>0 and testing[j,1]>0 else 0.0 for j in range(num_tests)])
if torch.cuda.is_available():
   testing = testing.to('cuda')
outputs = net(testing)
outputs=outputs.to('cpu')
#print(outputs)
correct = [1 if ((desired[j]>0.5 and outputs[j]>0.5) or (desired[j]<0.5 and outputs[j]<0.5)) else 0 for j in range(num_tests)]
print(sum(correct),len(correct),sum(correct)/len(correct))

tensor([[-30., -65.],
        [-30.,  47.],
        [ 60., -72.],
        ...,
        [ 43.,  -7.],
        [-33.,  38.],
        [-80., -53.]])
1995 2000 0.9975
